In [1]:
%%writefile ScribleSorcer.py
import streamlit as st
import numpy as np
from scipy import ndimage
from skimage import color, transform, img_as_ubyte, img_as_float
from skimage.filters import threshold_otsu, unsharp_mask
from skimage.measure import regionprops, label
from skimage.morphology import binary_opening, square, binary_erosion, binary_closing, binary_dilation
from skimage.transform import rotate
from PIL import Image
from joblib import load
from math import floor, ceil

# Load SVC model
svc_pipeline = load('best_model.joblib')

def preprocess_image(image_data):
    
    
    # Convert to grayscale
    grayscale_image = color.rgb2gray(image_data)
    
    #sharpen an image
    sharpened_image = unsharp_mask(grayscale_image, radius=1, amount=6)
    #st.image(img_as_ubyte(sharpened_image), caption='Sharpened', use_column_width=True)
    st.session_state.sharpened_image = sharpened_image
   
    # Apply thresholding
    thresh = threshold_otsu(sharpened_image)
    binary_image = sharpened_image > thresh
    
    # Invert the image to match MNIST format
    inverted_binary_image = img_as_ubyte(np.invert(binary_image))
    #st.image(inverted_binary_image, caption='Inverted', use_column_width=True)
    st.session_state.inverted_binary_image = inverted_binary_image

    # Apply morphological opening to clean up the image   
    opened_image = binary_opening(inverted_binary_image, square(5))
    #st.image(img_as_ubyte(opened_image), caption='open', use_column_width=True)
    st.session_state.opened_image = img_as_ubyte(opened_image)
    
    closed_image = binary_closing(opened_image, square(3))
    #st.image(img_as_ubyte(closed_image), caption='closed', use_column_width=True)
    st.session_state.closed_image = closed_image

    eroded_image = binary_erosion(closed_image, square(3))
    #st.image(img_as_ubyte(eroded_image), caption='erosion', use_column_width=True)
    st.session_state.eroded_image = eroded_image

    dilated_image = binary_dilation(eroded_image, square(5))
    #st.image(img_as_ubyte(dilated_image), caption='dilated', use_column_width=True)
    st.session_state.dilated_image = dilated_image
    
    # Label connected components
    labeled_img = label(dilated_image)
    regions = regionprops(labeled_img)
    
    # Assume the largest connected component is the digit
    digit_region = max(regions, key=lambda r: r.area)
    min_row, min_col, max_row, max_col = digit_region.bbox
    
    # Crop the digit using the bounding box
    cropped_digit = dilated_image[min_row:max_row, min_col:max_col]
    
    # Estimate the orientation and rotate the digit to be upright
    orientation = digit_region.orientation
    rotated_digit = rotate(cropped_digit, -np.degrees(orientation), resize=True)
    
    # Resize the digit to 20x20 while preserving the aspect ratio
    scale_factor = 20.0 / max(rotated_digit.shape)
    new_height = int(rotated_digit.shape[0] * scale_factor)
    new_width = int(rotated_digit.shape[1] * scale_factor)
    resized_digit = transform.resize(rotated_digit, (new_height, new_width), 
                                     anti_aliasing=False, order=0, mode='constant', cval=0, preserve_range=True)
    
    resized_digit_normalized = img_as_float(resized_digit)
    centered_digit_ubyte = img_as_ubyte((resized_digit_normalized - resized_digit_normalized.min())
                                        / (resized_digit_normalized.max() - resized_digit_normalized.min()))
    
    width_padding_total = 28 - new_width
    height_padding_total = 28 - new_height

    #then place padding around the image
    left_padding = floor(width_padding_total / 2)
    right_padding = ceil(width_padding_total / 2)
    top_padding = floor(height_padding_total / 2)
    bottom_padding = ceil(height_padding_total / 2)


    final_image = np.pad(centered_digit_ubyte,
                         ((top_padding, bottom_padding), (left_padding, right_padding)),
                         mode='constant', constant_values=0)   
    
    
    return final_image.flatten(), final_image

def predict(flattened_image):
    reshaped_image = flattened_image.reshape(1, -1)
    prediction = svc_pipeline.predict(reshaped_image)
    return prediction[0]
#--------------------------------------------------------------------------------------------------------
def capture():
    st.title('The Enchanted Numerologist: Guessing Your Scribbles!')
    st.markdown("""
    Welcome to the Enchanted Numerologist! Snap a picture of a digit scribble, and let the magic
    unfold as we see through the crystal ball the number you've drawn. Is it sorcery? 
    Or is it machine learning at work?
    Embark on a mystical journey and uncover the magic hidden within numbers!
    """)
    captured_image = st.camera_input("Capture a Scribble", label_visibility='hidden')

    if captured_image is not None:
        image = Image.open(captured_image)
        image_array = np.array(image)
              
        
        flattened_image, final_image_ubyte = preprocess_image(image_array)
        #st.image(final_image_ubyte, caption='Processed Image', use_column_width=True)
        st.session_state.final_image_ubyte = final_image_ubyte
        
        st.markdown("""
                    <div style='font-size: 30px; text-align: center;'>
                    🔮 Gazing into the crystal ball... 🔮
                    </div>
                    """, unsafe_allow_html=True)

        st.markdown("""
                    <div style='font-size: 26px; text-align: center;'>
                    Say AbraKaDabra and Click on Scribble Sorcery.
                    </div>
                    """, unsafe_allow_html=True)

        prediction = predict(flattened_image)
        #st.write(f'Predicted digit: {prediction}')
        st.session_state.prediction = prediction

def predict_page():
    st.title("Scribble Sorcery")
    prediction = st.session_state.prediction
    st.write(
        f"<div style='font-size: 22px; text-align: center;'>"
        f"The enchanted symbols illuminate... 🔮✨ "
        f"Behold, it unveils... : </div>"
        f"<div style='font-size: 110px; color: orange; text-align: center;'>{prediction}</div>",
        unsafe_allow_html=True
    )
    
    st.markdown("""
               <div style='font-size: 22px; text-align: center;'>
                Was the ethereal vision unveiled by the enchanted scribbler in harmony with 
                the cosmic resonance of the mystic realm?
                </div>
                 """, unsafe_allow_html=True)
    col1, col2 = st.columns(2)
    with col1:
        if st.button('Yes'):
            st.success("It's magic! 🌟")
    with col2:
        if st.button('No'):
            st.markdown("""
                    <div style='font-size: 22px;'>
                    ✨ Ah, it seems our magical essence is momentarily diminished! 
                    Not to worry, the arcane forces recharge with every attempt. 
                    Try casting your spell (take a new picture) once more, 
                    and let the mystical energies align for a clearer vision. ✨
                    </div>
                    """, unsafe_allow_html=True)
    
def behind_the_scenes_page():
    st.title("Sorcerer's Scrolls")
    st.markdown("""
    Welcome to the Sorcerer's Scrolls! Here, you will uncover the secrets behind the mystical
    predictions made by the Enchanted Numerologist. Stay tuned as we reveal the ancient spells
    and magical algorithms that power the enchanted predictions!
    """)
    
    sharpened_image = st.session_state.sharpened_image
    st.image(sharpened_image, caption='Sharpened')
    
    inverted_binary_image = st.session_state.inverted_binary_image
    st.image(inverted_binary_image, caption='Inverted Binary')
    
    opened_image = st.session_state.opened_image 
    st.image(opened_image, caption='Open')
    
    closed_image = st.session_state.closed_image
    st.image(opened_image, caption='Closed')
    
    eroded_image = st.session_state.eroded_image 
    st.image(opened_image, caption='Eroded')
    
    dilated_image = st.session_state.dilated_image
    st.image(opened_image, caption='Dilated')    
        
    final_image_ubyte = st.session_state.final_image_ubyte 
    st.image(final_image_ubyte, caption='Final', use_column_width=True)
    
    
    
def main():
    st.sidebar.title("Mystical Menu")
    choice = st.sidebar.radio("Choose an enchantment:", ["Commence the Enchantment", "Scribble Sorcery", "Sorcerer's Scrolls"])

    if choice == "Commence the Enchantment":
        st.session_state.capture_done = False

    if choice == "Commence the Enchantment":
        capture()
    elif choice == "Scribble Sorcery":
        predict_page()
    elif choice == "Sorcerer's Scrolls":
        behind_the_scenes_page()

if __name__ == "__main__":
    main()


Overwriting ScribleSorcer.py


In [ ]:
!streamlit run ScribleSorcer.py